## Objective:
- Import required libraries data pre-processing; numpy, pyarrow
- Convert Entire OCO2/OCO3 netCDF files to parquet format

#### Why Parquet format?
Parquet is an open source file format built to handle flat columnar storage data formats. Parquet operates well with complex data in large volumes.It is known for its both performant data compression and its ability to handle a wide variety of encoding types. 

- Parquet deploys Google's record-shredding and assembly algorithm that can address complex data structures within data storage. Some Parquet benefits include:

    * Fast queries that can fetch specific column values without reading full row data
    * Highly efficient column-wise compression
    * High compatibility with with OLAP
    
#### Benifits of using Parquet format over CSV
- HOW IS PARQUET DIFFERENT FROM CSV?

While CSV is simple and the most widely used data format (Excel, Google Sheets), there are several distinct advantages for Parquet, including:

- Parquet is column oriented and CSV is row oriented. Row-oriented formats are optimized for OLTP workloads while column-oriented formats are better suited for analytical workloads.
- Column-oriented databases such as AWS Redshift Spectrum bill by the amount data scanned per query

Therefore, converting CSV to Parquet with partitioning and compression lowers overall costs and improves performance

[source: Snowflake](https://www.snowflake.com/guides/what-parquet#:~:text=Parquet%20is%20an%20open%20source,wide%20variety%20of%20encoding%20types.)

In [ ]:
# !pip install pyarrow

In [1]:
import os
import numpy as np
import pandas as pd
import pyarrow as pa
import netCDF4 as nc

from pyarrow import parquet as parq
from datetime import datetime

In [ ]:
## Example: creating a parquet file using table using the array
arr= np.arange(1.0, 20.0)
pa_table= pa.table({"float_data": arr})
parq.write_table(pa_table, 'test_table_data.parquet')

### NETCDF files
- PATH: downloaded netCDF files from the source
- EG: # list fo FILES initially downloaded locally

In [ ]:
# path_= '../../../Cluster_machine/OCO2/B_11_new_version/2020/'
path_="../../../Cluster_machine/OCO3/2021/"

In [ ]:
file_path_= []

for root, dirs, files in os.walk(path_):
    
    for filename in files:
        #print(os.path.join(root, filename))
        
        # Append the files into list
        file_path_.append(os.path.join(root, filename))
        
file_path_[:3]

In [ ]:
file_names= file_path_

## conv dateTime

In [ ]:
def conv_date(d):
    return datetime.strptime(str(d), '%Y%m%d%H%M%S%f')

In [ ]:
%%time
countFiles=0

lon_list= []
lat_list= []
xco2_list= []
qual_flag_list= []
lat=[]
lon=[]
dateTime= []

for j in file_names:
    if j.endswith(".nc4"):
        var_= nc.Dataset(j)
        lon_list.append(np.array(var_.variables['vertex_longitude'][:]).tolist())
        lat_list.append(np.array(var_.variables['vertex_latitude'][:]).tolist())
        xco2_list.append(np.array(var_.variables['xco2'][:]).tolist())
        qual_flag_list.append(np.array(var_.variables['xco2_quality_flag'][:]).tolist())
        
        # DateTIme formating
        dateTime.append(np.array(var_.variables['sounding_id'][:].tolist()))

        # lat and long without vert
        lat.append(np.array(var_.variables['latitude'][:].tolist()))
        lon.append(np.array(var_.variables['longitude'][:].tolist()))

#print('\nTotalFiles: ', countFiles)

In [ ]:
%%time
lon_list_a= [element for sublist in lon_list for element in sublist]
lat_list_a= [element for sublist in lat_list for element in sublist]
xco2_list_a= [element for sublist in xco2_list for element in sublist]
lon_a= [element for sublist in lon for element in sublist]
lat_a= [element for sublist in lat for element in sublist]
xco2_qual_flag= [ element for sublist in qual_flag_list for element in sublist]
dateTime_list= [ element for sublist in dateTime for element in sublist]

### Transformation to parquet format
- Creating table format

In [ ]:
%%time
df_oco3= pa.table({
    'Latitude_vertices': lat_list_a,
    'Longitude_vertices': lon_list_a,
    'Latitude': lat_a,
    'Longitude': lon_a,
    'Xco2': xco2_list_a,
    'quality_flag': xco2_qual_flag,
    'DateTime': dateTime_list
})

## Table from parquet

In [ ]:
# 'oco3_20222_parq.parquet'
file_name= input("File Name")

In [ ]:
%%time
parq.write_table(df_oco3, file_name)

# Pre-processing the original file
- CONVERT the DATETIME -> Date and Month
- Filter by quality flag
    - Good Quality-> 0

In [2]:
%%time
df_parq= pd.read_parquet("oco3_2022_parq.parquet", engine="pyarrow")
df_parq.head(2)

Wall time: 18.6 s


,Latitude_vertices,Longitude_vertices,Latitude,Longitude,Xco2,quality_flag,DateTime
0,"[33.45766067504883, 33.44253921508789, 33.4476...","[130.08013916015625, 130.09841918945312, 130.1...",33.453053,130.099274,421.111877,1,2022010100011738
1,"[33.340702056884766, 33.32762145996094, 33.332...","[130.1199951171875, 130.1375732421875, 130.158...",33.338490,130.138748,424.939941,1,2022010100011974


## Quality Flag filtering:

In [3]:
%%time
df_qual_= df_parq[df_parq["quality_flag"]==0]

Wall time: 1.39 s


In [4]:
len(df_parq), len(df_qual_)

(38260867, 17984969)

# Convert the DATETIME format

In [5]:
%%time
df_qual_["DateTime"]= pd.to_datetime(df_qual_["DateTime"], format="%Y%m%d%H%M%S%f")

Wall time: 39.2 s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
df_qual_= df_qual_.reset_index()
df_qual_=df_qual_.drop(columns=["index"])
df_qual_

,Latitude_vertices,Longitude_vertices,Latitude,Longitude,Xco2,quality_flag,DateTime
0,"[33.314796447753906, 33.30024337768555, 33.308...","[130.20492553710938, 130.2227783203125, 130.24...",33.311535,130.222916,418.262268,0,2022-01-01 00:01:40.060
1,"[33.27663803100586, 33.262542724609375, 33.270...","[130.16879272460938, 130.1868896484375, 130.20...",33.273293,130.186539,418.319977,0,2022-01-01 00:01:40.330
2,"[33.28447723388672, 33.27030944824219, 33.2781...","[130.1868896484375, 130.2049560546875, 130.222...",33.281330,130.204788,416.698120,0,2022-01-01 00:01:40.340
3,"[33.292598724365234, 33.27840805053711, 33.285...","[130.205322265625, 130.22335815429688, 130.240...",33.289188,130.222794,417.723663,0,2022-01-01 00:01:40.350
4,"[33.30021286010742, 33.28603744506836, 33.2941...","[130.22283935546875, 130.2408447265625, 130.25...",33.297165,130.240952,417.152893,0,2022-01-01 00:01:40.360
...,...,...,...,...,...,...,...
17984964,"[-4.618454933166504, -4.635359764099121, -4.62...","[168.7618408203125, 168.77386474609375, 168.78...",-4.624123,168.775085,416.166016,0,2022-10-30 23:57:22.020
17984965,"[-4.612674713134766, -4.629579067230225, -4.62...","[168.77703857421875, 168.7890625, 168.80322265...",-4.618398,168.790131,416.159760,0,2022-10-30 23:57:22.030
17984966,"[-4.607049942016602, -4.623953819274902, -4.61...","[168.791748046875, 168.80377197265625, 168.818...",-4.612716,168.804993,416.555115,0,2022-10-30 23:57:22.040
17984967,"[-4.674790859222412, -4.691671371459961, -4.68...","[168.8399658203125, 168.85198974609375, 168.86...",-4.680434,168.853180,416.147766,0,2022-10-30 23:57:23.340


In [7]:
%%time
df_qual_.to_parquet("oco3_2022_parq_dateTime.parquet", engine="pyarrow")

Wall time: 14.1 s
